In [1]:
cd ..

C:\Users\User\Documents\SURF 2021\RL RNN SURF\src


We first define the input data. The input data is has the following structure: We have a group of subjects (say S1 and S2) and each subject has completed the decision-making task multiple times (multiple blocks), say each subject has completed the task two times in this example, i.e., we have two blocks of data for each subject. The data within each block is a ditonary containing three numpy arrays: 'action', 'state', 'reward'. 

'action' containig the actions taken by the subject on each trial and it should be a non-zero integer or -1. If the action is -1 it will coded by a zero vector and corresponds to no-action. Dimesionlity of 'action' is B x |T| in which |T| is the number of trials.

'state' contains the state of the environment each trial. Its dimesionlity is B x |T| x |S| in which |T| is the number of trials, and |S| is the lenght of state vector. 

'reward' contains the reward received after taking each actions. Its dimesionlity is B x |T| in which |T| is the number of trials.

For example, if subject S1 has completed 6 trials in the firt block and 4 trials in the second block and subject 2 has completed 5, 6 trials in the first and second blocks respectivly, then the data structure can look like this:

   

In [2]:
import pandas as pd

data = pd.read_csv("../data/BD/choices_diagno.csv.zip", compression='zip', header=0, sep=',', quotechar='"')
data['reward'] = [0 if x == 'null' else 1 for x in data['outcome']]
data['id'] = data['ID']
data['block'] = data['trial']

#R1: right, R2: Left
data['action'] = [0 if x == 'R1' else 1 for x in data['key']]
del data['trial']
del data['ID']
data

,Unnamed: 0,key,block,time,condition,outcome,reward,code,choice,diag,PressRate.sec,Mania,Depression,id,action
0,1,R2,1,4981.000,6,NaN,1,2,R2,Depression,1.45,1,13,s_005_,1
1,2,R1,1,5572.000,6,O2,1,3,R1,Depression,1.45,1,13,s_005_,0
2,3,R1,1,6917.000,6,NaN,1,1,R1,Depression,1.45,1,13,s_005_,0
3,4,R2,1,7274.000,6,NaN,1,2,R2,Depression,1.45,1,13,s_005_,1
4,5,R1,1,7707.000,6,NaN,1,1,R1,Depression,1.45,1,13,s_005_,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132246,132247,R2,12,1780.489,1,NaN,1,1,R1,Healthy,1.43,0,0,s_9_da,1
132247,132248,R2,12,1780.771,1,O1,1,3,R1,Healthy,1.43,0,0,s_9_da,1
132248,132249,R2,12,1781.817,1,NaN,1,1,R1,Healthy,1.43,0,0,s_9_da,1
132249,132250,R2,12,1782.097,1,NaN,1,1,R1,Healthy,1.43,0,0,s_9_da,1


In [33]:
import pandas as pd
import numpy as np

data = pd.DataFrame()
for i in range(1,101):
    new_data = pd.read_csv('simulationData_beta10_halfhalf\sim_'+str(i)+'.csv', header=0, sep=',', quotechar='"')
    new_data['id'] = new_data['subID']
    new_data['block'] = new_data['blockID']
    new_data['reward'] = new_data['outcome']
    new_data['action'] = [x-1 for x in new_data['respKey']]
    new_data['diag'] = ['Novelty' if i <= 50 else 'Uncertainty' for x in new_data.index]
    new_data['state'] = [[x,y] for (x,y) in zip(new_data['trialStimID_1'],new_data['trialStimID_2'])]
    del new_data['subID']
    del new_data['blockID']
    del new_data['outcome']
    del new_data['respKey']
    del new_data['trialStimID_1']
    del new_data['trialStimID_2']
    data = data.append(new_data, ignore_index=True)
data

,trialID,RT,selectedStimID,workerId,trialStimReward_1,trialStimReward_2,isTrialStim_1,isTrialStim_2,isTrialStim_3,isTrialStim_4,...,lossHistoryStim_1,lossHistoryStim_2,exposeHistoryStim_1,exposeHistoryStim_2,id,block,reward,action,diag,state
0,1,NaN,2,A3KKY9HXZRRAIV,1,0,0,1,0,1,...,NaN,NaN,NaN,NaN,1,1,1,0,Novelty,"[2, 4]"
1,2,NaN,2,A3KKY9HXZRRAIV,1,0,0,1,0,1,...,NaN,NaN,NaN,NaN,1,1,1,0,Novelty,"[2, 4]"
2,3,NaN,2,A3KKY9HXZRRAIV,1,0,0,1,0,1,...,NaN,NaN,NaN,NaN,1,1,1,0,Novelty,"[2, 4]"
3,4,NaN,3,A3KKY9HXZRRAIV,1,0,0,0,1,1,...,NaN,NaN,NaN,NaN,1,1,1,0,Novelty,"[3, 4]"
4,5,NaN,2,A3KKY9HXZRRAIV,1,0,0,1,0,1,...,NaN,NaN,NaN,NaN,1,1,1,0,Novelty,"[2, 4]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,11,NaN,13,A377LTGWJKY2IW,0,0,1,0,0,0,...,NaN,NaN,NaN,NaN,100,11,0,0,Uncertainty,"[13, 1]"
15996,12,NaN,4,A377LTGWJKY2IW,0,1,1,0,0,1,...,NaN,NaN,NaN,NaN,100,11,1,1,Uncertainty,"[1, 4]"
15997,13,NaN,4,A377LTGWJKY2IW,0,0,1,0,0,1,...,NaN,NaN,NaN,NaN,100,11,0,0,Uncertainty,"[4, 1]"
15998,14,NaN,13,A377LTGWJKY2IW,1,0,0,0,0,1,...,NaN,NaN,NaN,NaN,100,11,0,1,Uncertainty,"[4, 13]"


In [43]:
data.loc[data.diag == 'Novelty']['reward'].to_numpy()

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [9]:
#  Data
import numpy as np
import csv
data={}
test_data = {}
for i in range(1,101):
    with open('simulationData_beta10_halfnovelty_halfminusuncertainty\sim_'+str(i)+'.csv', 'r' ) as theFile:
        reader = csv.reader(theFile)
        headers = next(reader, None)
        actions = []
        states = []
        rewards = []
        for line in reader:
            actions.append(int(line[3]))
            states.append([int(line[8]),int(line[9])])
            rewards.append(int(line[5]))
        block = [
                {
                    'action': np.array([actions])-1,
                    'state': np.array([states]),
                    'reward': np.array([rewards]),
                    'id': 'S'+str(i),

                    'block': 0
                }

            ]
        if i<91:
            data['S'+str(i)] = block
        else:
            test_data['S'+str(i)] = block

In [10]:
data

{'S1': [{'action': array([[0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1,
           0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0,
           1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1,
           0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1,
           1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
           1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,
           1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,
           0, 0, 0, 0, 1, 1]]),
   'state': array([[[ 2,  4],
           [ 2,  4],
           [ 2,  4],
           [ 3,  4],
           [ 2,  4],
           [ 4,  3],
           [ 4,  3],
           [ 4,  2],
           [ 2,  3],
           [ 2,  3],
           [ 4,  2],
           [ 4,  3],
           [ 2,  4],
           [ 4,  2],
           [ 2,  3],
           [ 4,  2],
           [ 4, 10],
           [10,  4],
           [10,  4],
     

In the above example, |A|=2 (there are two actions coded as 0 and 1), and |S|=2 (the state vector has two elements). For example, if there are three stimuli in the environment, they can be coded as [1, 0, 0], [0, 1, 0], [0, 0, 1] state vectors. In this case |S|=3.

In [7]:
from actionflow.rnn.lstm_beh import LSTMBeh
worker = LSTMBeh(a_size=2, s_size=2, n_cells=20)

c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\user\documents\surf

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Finally, we train the model:

In [8]:
from actionflow.rnn.opt_beh import OptBEH
from actionflow.util.logger import LogFile

output_path = '../results/test/'
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, None,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None
                    )

2021-07-30 20:35:35,188 - DeepRL - DEBUG - version control: (None, None)
2021-07-30 20:35:35,188 - DeepRL - DEBUG - learning rate: 0.01
2021-07-30 20:35:35,188 - DeepRL - DEBUG - global iters: 50
2021-07-30 20:35:35,188 - DeepRL - DEBUG - training data-points: 16000
2021-07-30 20:35:35,197 - DeepRL - DEBUG - test data-points: None


Instructions for updating:
Use tf.cast instead.


2021-07-30 20:35:35,622 - DeepRL - DEBUG - opt started...


INFO:tensorflow:../results/test/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


TypeError: 'float' object is not subscriptable

In [ ]:
from actionflow.data.data_process import DataProcess
train_merged = DataProcess.merge_data(data)
train_merged

And then the merged data can be used for training the model as before. The test data can also be passed to the training method, in order to test the mmodel on the training data in regular intervals. Say the test data is as follows:

and we want to test the model every 10 iterations:

In [ ]:
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, test_data,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None,
                    test_period=10
                    )

In [ ]:
test_data["S100"]